# Mengimport Spark SQL dan membuat session

In [4]:
import findspark
findspark.init()
import pyspark

In [5]:
# mengimport modul yang dibutuhkan
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

# membuat SparkSession
appName = "Data Preprocessing di Apache Spark"
spark = SparkSession \
.builder \
.appName(appName) \
.config("spark.some.config.option", "some-value") \
.getOrCreate()

# Memuat dataset dari file

In [7]:
# membuat skema file
flightSchema = StructType([
StructField("DayofMonth", IntegerType(), False),
StructField("DayOfWeek", IntegerType(), False),
StructField("Carrier", StringType(), False),
StructField("OriginAirportID", IntegerType(), False),
StructField("DestAirportID", IntegerType(), False),
StructField("DepDelay", IntegerType(), False),
StructField("ArrDelay", IntegerType(), False),
])

# membaca data dari file ke DataFrame
flightDataFrame = spark.read.csv('raw-flight-data.csv', schema=flightSchema, header=True)
flightDataFrame.show(3)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 3 rows



In [9]:
# membuat skema file
airportSchema = StructType([
StructField("Airport_ID", IntegerType(), False),
StructField("City", StringType(), False),
StructField("State", StringType(), False),
StructField("Name", StringType(), False),
])

# membaca data dari file ke DataFrame
airportDataFrame = spark.read.csv('airports.csv', schema=airportSchema, header=True)
airportDataFrame.show(3)

+----------+-----------+-----+--------------------+
|Airport_ID|       City|State|                Name|
+----------+-----------+-----+--------------------+
|     10165|Adak Island|   AK|                Adak|
|     10299|  Anchorage|   AK|Ted Stevens Ancho...|
|     10304|      Aniak|   AK|       Aniak Airport|
+----------+-----------+-----+--------------------+
only showing top 3 rows



# Merge dua dataframe dan tampilkan jumlah penerbangan tiap kota

In [10]:
flightByOrigin = flightDataFrame.join(airportDataFrame,
                                     flightDataFrame.OriginAirportID ==
                                     airportDataFrame.Airport_ID).groupBy("City").count()
flightByOrigin.show(3)

+--------------+-----+
|          City|count|
+--------------+-----+
|       Phoenix|90281|
|         Omaha|13537|
|Raleigh/Durham|28436|
+--------------+-----+
only showing top 3 rows



# Handle duplicated data

In [11]:
# hitung jumlah baris data
n1 = flightDataFrame.count()
print("Jumlah baris data pada DataFrame Flight: ", n1)

# hitung jumlah baris data setelah menghapus data duplikat
n2 = flightDataFrame.dropDuplicates().count()
print("Jumlah baris data setelah data duplikat dihapus: ", n2)
n3 = n1 - n2
print("Jumlah baris data duplikat pada DataFrame Flight: ", n3)

Jumlah baris data pada DataFrame Flight:  2719418
Jumlah baris data setelah data duplikat dihapus:  2696983
Jumlah baris data duplikat pada DataFrame Flight:  22435


In [13]:
# Kita juga bisa membuat kriteria untuk data duplikat berdasar kolom
import pandas as pd

df_pd = pd.DataFrame(
    data={'name': ['Rony','Rony','Rony'],
     'age': [27,15,27],
     'height': [168,165,168]}
)

df = spark.createDataFrame(df_pd)
df.show()
df.dropDuplicates().show()

+----+---+------+
|name|age|height|
+----+---+------+
|Rony| 27|   168|
|Rony| 15|   165|
|Rony| 27|   168|
+----+---+------+

+----+---+------+
|name|age|height|
+----+---+------+
|Rony| 27|   168|
|Rony| 15|   165|
+----+---+------+



In [14]:
df.dropDuplicates(['name']).show()

+----+---+------+
|name|age|height|
+----+---+------+
|Rony| 27|   168|
+----+---+------+



# Handle missing data

In [15]:
# Hapus baris jika ada setidaknya satu kolom kosong
flightsNoMissingValue = flightDataFrame.dropDuplicates().dropna(
    how="any", subset=["ArrDelay", "DepDelay"]) # pakai how="all" apabila semua kolom ada yang kosong
numberOfMissingValueAny = n1 - flightsNoMissingValue.count()
print("Jumlah baris yang memiliki data kosong: ", numberOfMissingValueAny)

Jumlah baris yang memiliki data kosong:  46233


In [16]:
# Mengisi data yang kosong dengan nilai rata-rata pada kolom tersebut
meanArrDelay = flightDataFrame.groupBy().avg("ArrDelay").take(1)[0][0]
print("Rata-rata kolom ArrDelay: ", meanArrDelay)
meanDepDelay = flightDataFrame.groupBy().avg("DepDelay").take(1)[0][0]
print("Rata-rata kolom DepDelay: ", meanDepDelay)

# menghapus data duplikat dan mengisi data yang kosong dengan nilai rata-rata pada kolom tersebut
flightsCleanData = flightDataFrame.fillna(
    {'ArrDelay': meanArrDelay, 'DepDelay': meanDepDelay})

Rata-rata kolom ArrDelay:  6.63768791455498
Rata-rata kolom DepDelay:  10.53686662649788


# Menampilkan statistik dari dataset

In [17]:
flightsCleanData.describe('DepDelay','ArrDelay').show()

+-------+------------------+-----------------+
|summary|          DepDelay|         ArrDelay|
+-------+------------------+-----------------+
|  count|           2719418|          2719418|
|   mean|10.531448640848888|6.630879842672218|
| stddev| 35.91695039008144|38.44200618946895|
|    min|               -63|              -94|
|    max|              1863|             1845|
+-------+------------------+-----------------+



In [18]:
# melihat nilai korelasi dari kolom DepDelay dan ArrDelay
correlation = flightsCleanData.corr('DepDelay', 'ArrDelay')
print("Nilai korelasi kolom DepDelay dan ArrDelay: ", correlation)

Nilai korelasi kolom DepDelay dan ArrDelay:  0.9393538215572638
